<a href="https://colab.research.google.com/github/Sabindawadi12/Python_Takeo/blob/Projects/Airbnb_ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AirBNB data/AB_NYC_2019.csv')

In [ ]:
#need to drop 10 rows, where price == 0
drop_me1 = df[(df['neighbourhood_group'] == 'Manhattan') & (df['price'] == 0)].index
drop_me2 = df[(df['neighbourhood_group'] == 'Brooklyn') & (df['price'] == 0)].index
drop_me3 = df[(df['neighbourhood_group'] == 'Bronx') & (df['price'] == 0)].index

#drop and override
df = df.drop(drop_me1)
df = df.drop(drop_me2)
df = df.drop(drop_me3)

df.shape

(48884, 16)

In [ ]:
#missing values in categorical variables
missing_values = df.isna().sum()
print(missing_values)

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10051
reviews_per_month                 10051
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [ ]:
df.drop(['name', 'id', 'host_name', 'host_id','last_review' ], axis =1, inplace = True)

In [ ]:
# Calculate the mean of the 'reviews_per_month' column
mean_reviews_per_month = df['reviews_per_month'].mean()

# Replace NaN values with the mean
df['reviews_per_month'].fillna(mean_reviews_per_month, inplace=True)

In [ ]:
df.head(3)

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.210000,6,365
1,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.380000,2,355
2,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,1.373128,1,365


In [ ]:
#Dependent , independent
y = df.iloc[:, 5:6]

X= df.iloc[:, :5].join(df.iloc[:, 6:]) #except i=5 column
print(type(X))

<class 'pandas.core.frame.DataFrame'>


Categorical encoding

#label encoding
room type (i=4). Why? order matters!

neighborhood (i=1). Why? too many categories, so,don't use dummies.

#onehot ecnoding
neighbourhood_group (i=0). Why? order doesn't matter & few categories



In [ ]:
#categorical encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Specify the column indices to encode
column_indices = [ 0,1, 4]

#use label encoder and override
X.iloc[:, column_indices]= X.iloc[:, column_indices].apply(encoder.fit_transform) #direct override, no need to concatenate

print(X)

       neighbourhood_group  neighbourhood  latitude  longitude  room_type  \
0                        1            108  40.64749  -73.97237          1   
1                        2            127  40.75362  -73.98377          0   
2                        2             94  40.80902  -73.94190          1   
3                        1             41  40.68514  -73.95976          0   
4                        2             61  40.79851  -73.94399          0   
...                    ...            ...       ...        ...        ...   
48890                    1             13  40.67853  -73.94995          1   
48891                    1             28  40.70184  -73.93317          1   
48892                    2             94  40.81475  -73.94867          0   
48893                    2             95  40.75751  -73.99112          2   
48894                    2             95  40.76404  -73.98933          1   

       minimum_nights  number_of_reviews  reviews_per_month  \
0           

<ipython-input-11-250662d2c9df>:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.iloc[:, column_indices]= X.iloc[:, column_indices].apply(encoder.fit_transform) #direct override, no need to concatenate


In [ ]:
# # from sklearn.preprocessing import OneHotEncoder
# # encoder = OneHotEncoder()
# # column_index = [0]



# # # Extract the column to be encoded as a separate DataFrame
# column_to_encode = X.iloc[:, column_index]


# # # Apply one-hot encoding to the extracted columns and convert it into data frame
# encoded_column = pd.DataFrame(encoder.fit_transform(column_to_encode).toarray())

# # drop one dummy to avoid perfect multicollinearity
# encoded_column = encoded_column.drop(encoded_column.columns[0], axis=1)

# #Drop the original column from the original DataFrame
# X = X.drop(columns=[column_to_encode.name])

# # # Concatenate the encoded DataFrame with the original DataFrame
# # X = pd.concat([encoded_df, X], axis=1)

# # # Print the updated DataFrame
# # print(X)


In [ ]:
X.shape
y.values.ravel().shape

(48884,)

In [ ]:
#train test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Apply the scaler to transform the training and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimator=10)
rf_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test_scaled)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)

TypeError: ignored

In [ ]:
#to check important features for RF
print(rf_classifier.feature_importances_)

In [ ]:
#May be not logistic regression?
from sklearn.metrics import accuracy_score
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = logreg.predict(X_test_scaled)

# Calculate the accuracy of the model
accuracy_lr = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy of logistic:", accuracy_lr)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score

# Create the ANN model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(X_train_scaled, y_train, epochs=2, batch_size=32)

# Make predictions on the test data
y_pred = model.predict(X_test_scaled)

# Calculate the accuracy of the model
accuracy_ann = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy_ann)

Epoch 1/2
1223/1223 [==============================] - 6s 3ms/step - loss: -1765613.0000 - accuracy: 0.0000e+00
Epoch 2/2
306/306 [==============================] - 1s 2ms/step
Accuracy: 0.0


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_classifier = SVC()
svm_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = svm_classifier.predict(X_test_scaled)

# Calculate the accuracy of the model
accuracy_svm = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy_svm)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: ignored